In [14]:
import pandas as pd

filt_spec_genes = False
for i, suffix in enumerate(['m1', 'm2', 'm3']):
    # filename = './Data/gr_Div_DG_'+ suffix+'_standardized_ByNoDrug.csv'
    filename = './Data/gr_DG_'+ suffix+'_normalized.csv'
    df = pd.read_csv(filename, index_col='Gene_inhibition').add_suffix('_'+suffix)
    if i ==0:
        result = df.filter(regex='^po_|^First_gene|^Second_gene')
    else:
        result = result.merge(df.filter(regex='^po_'), left_index=True, right_index=True)
spec_genes = ['dadX','pheA','acnB','mrcA','dapF','gnd']
if filt_spec_genes == True:
    result = result.query('First_gene_m1 not in @spec_genes and Second_gene_m1 not in @spec_genes')
result = result.iloc[:,2:] # E only

In [17]:
import pandas as pd
import itertools 
from statsmodels.stats.contingency_tables import mcnemar
pd.set_option('display.max_rows', 150)

def classify_drug_response(s, additive_threshold_anta=.01, additive_threshold_syn=.01):

    # diff_bins = [-2000, -1*additive_threshold,1*additive_threshold,10]
    # return pd.cut(s, bins=diff_bins, labels=['Antagonistic', 'Additive', 'Synergistic'])
    diff_bins = [-2000, -10,-1*additive_threshold_anta,1*additive_threshold_syn,10]
    return pd.cut(s, bins=diff_bins, labels=['Drug_comb_enable_growth','Antagonistic', 'Additive', 'Synergistic'])

def get_contingency_table(pairwise_options , additive_threshold_anta=.01, additive_threshold_syn=.01):
    result_classified = result.apply(classify_drug_response)

    all_tables = [pd.DataFrame(index=['Drug_comb_enable_growth','Antagonistic', 'Additive', 'Synergistic'], columns=['Drug_comb_enable_growth','Antagonistic', 'Additive', 'Synergistic'])
    ]

    if pairwise_options == 'method':
        another_group = 'Culture_item'
        group_element_list = ['coculture', 'monoculture']
        another_group_element_list = ['m1','m3','m2'] 
        element_position = -1
    else:
        another_group = 'methods'
        group_element_list = ['m1','m2','m3'] 
        another_group_element_list = ['coculture', 'monoculture']
        element_position = -2

    all_items = ['_.*'.join(ele) for ele in list(itertools.product(['E0', 'S0'], group_element_list))]

    for items in all_items: 
        check_table = result_classified.filter(regex=items)
        check_table.columns = [col.split('_')[element_position] for col in check_table.columns]

        # for col1, col2 in itertools.combinations([check_table.columns], 2):
        for col1, col2 in itertools.combinations(another_group_element_list, 2):
            ct = pd.crosstab(index=check_table[col1], columns=check_table[col2])
            
            bunch = mcnemar(ct)
            p_value = bunch.pvalue
            # score = ((ct - expected) ** 2 / expected).sum()
            
            ct[f'Pairwise_{pairwise_options}(row-col)'] = str(col1) + '-' + str(col2)
            ct[another_group] = items
            ct['p_value'] = p_value
            
            all_tables.append(ct)

        # result_classified.filter(regex=items).sort_values(by=f'po_diff_{items}_m1')
    concatenated_table = pd.concat(all_tables)
    return concatenated_table, another_group

def generate_contingency_table(pairwise_options='method'):

    concatenated_table, another_group = get_contingency_table(pairwise_options)
    tab_cult = (concatenated_table.drop('Drug_comb_enable_growth', axis=1)
                .set_index([f'Pairwise_{pairwise_options}(row-col)','p_value'
                            , another_group], append=True)).swaplevel(0, 3).iloc[4:]
    print('pairwise culture comparison') if pairwise_options == 'culture' else print('pairwise method comparison')
    return tab_cult

# difference by method

In [18]:
generate_contingency_table(pairwise_options='method')

pairwise method comparison


Antagonistic  \
Culture_item     Pairwise_method(row-col) p_value                              
E0_.*coculture   m1-m3                    1.000000 Antagonistic            4   
                                                   Additive                0   
                                                   Synergistic             0   
                 m1-m2                    1.000000 Antagonistic           10   
                                                   Additive                1   
                                                   Synergistic             7   
                 m3-m2                    1.000000 Antagonistic            4   
                                                   Additive                1   
                                                   Synergistic            13   
E0_.*monoculture m1-m3                    1.000000 Antagonistic           17   
                                                   Additive                0   
                                                   Synergistic             0   
                 m1-m2                    0.003906 Antagonistic            8   
                                                   Additive                0   
                                                   Synergistic             0   
                 m3-m2                    0.003906 Antagonistic            8   
                                                   Additive                0   
                                                   Synergistic             0   
S0_.*coculture   m1-m3                    1.000000 Antagonistic            3   
                                                   Additive                0   
                                                   Synergistic             0   
                 m1-m2                    0.500000 Antagonistic            9   
                                                   Additive                2   
                                                   Synergistic             7   
                 m3-m2                    0.500000 Antagonistic            3   
                                                   Additive                2   
                                                   Synergistic            13   
S0_.*monoculture m1-m3                    1.000000 Antagonistic           19   
                                                   Additive                0   
                                                   Synergistic             0   
                 m1-m2                    0.070312 Antagonistic           12   
                                                   Additive                1   
                                                   Synergistic             0   
                 m3-m2                    0.070312 Antagonistic           12   
                                                   Additive                1   
                                                   Synergistic             0   

                                                                Additive  \
Culture_item     Pairwise_method(row-col) p_value                          
E0_.*coculture   m1-m3                    1.000000 Antagonistic        0   
                                                   Additive            2   
                                                   Synergistic         0   
                 m1-m2                    1.000000 Antagonistic        0   
                                                   Additive            1   
                                                   Synergistic         1   
                 m3-m2                    1.000000 Antagonistic        0   
                                                   Additive            1   
                                                   Synergistic         1   
E0_.*monoculture m1-m3                    1.000000 Antagonistic        0   
                                                   Additive           12   
                                   

# difference by monoculture and coculture

In [19]:
generate_contingency_table(pairwise_options='culture')

pairwise culture comparison


Antagonistic Additive  \
methods Pairwise_culture(row-col) p_value                                       
E0_.*m1 coculture-monoculture     0.625000 Antagonistic            7        3   
                                           Additive                1        1   
                                           Synergistic             9        8   
E0_.*m2 coculture-monoculture     0.007385 Antagonistic            4       13   
                                           Additive                2        0   
                                           Synergistic             2        8   
E0_.*m3 coculture-monoculture     1.000000 Antagonistic            4        0   
                                           Additive                1        1   
                                           Synergistic            12       11   
S0_.*m1 coculture-monoculture     0.250000 Antagonistic            9        0   
                                           Additive                3        0   
                                           Synergistic             7        9   
S0_.*m2 coculture-monoculture     0.022461 Antagonistic            7       11   
                                           Additive                2        0   
                                           Synergistic             4        5   
S0_.*m3 coculture-monoculture     0.250000 Antagonistic            3        0   
                                           Additive                3        0   
                                           Synergistic            13        9   

                                                        Synergistic  
methods Pairwise_culture(row-col) p_value                            
E0_.*m1 coculture-monoculture     0.625000 Antagonistic           0  
                                           Additive               1  
                                           Synergistic           20  
E0_.*m2 coculture-monoculture     0.007385 Antagonistic           1  
                                           Additive               0  
                                           Synergistic           20  
E0_.*m3 coculture-monoculture     1.000000 Antagonistic           0  
                                           Additive               0  
                                           Synergistic           21  
S0_.*m1 coculture-monoculture     0.250000 Antagonistic           1  
                                           Additive               0  
                                           Synergistic           21  
S0_.*m2 coculture-monoculture     0.022461 Antagonistic           0  
                                           Additive               0  
                                           Synergistic           21  
S0_.*m3 coculture-monoculture     0.250000 Antagonistic           0  
                                           Additive               0  
                                           Synergistic           22

In [20]:
!jupyter nbconvert --to html compare_drug_response_3methods.ipynb

[NbConvertApp] WARNING | Config option `extra_template_paths` not recognized by `HTMLExporter`.  Did you mean `template_path`?
[NbConvertApp] Converting notebook compare_drug_response_3methods.ipynb to html
[NbConvertApp] Writing 301740 bytes to compare_drug_response_3methods.html
